In [3]:
pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.


In [35]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn import preprocessing
from tabulate import tabulate

In [5]:
### 1. Load the data
data_features = pd.read_csv('../dengue_fever/data/dengue_features_train.csv')
data_labels = pd.read_csv('../dengue_fever/data/dengue_labels_train.csv')
data = pd.merge(data_features, data_labels, on=['city', 'year', 'weekofyear'])
data_features_test = pd.read_csv('../dengue_fever/data/dengue_features_test.csv')

In [6]:
### 2. Missing Values

# Perform linear interpolation where there is missing data
data = data.interpolate()
data_features_test = data_features_test.interpolate()

# Remove outliers (they have lot of missing data)
outliers_sj = [89, 141, 401, 453, 713, 765]
outliers_iq = [184, 236, 444, 496]
outliers = outliers_sj + [x + 936 for x in outliers_iq]
data = data.drop([x - 2 for x in outliers]) # Because element index is element - 2

In [7]:
### 3. Feature Selection

features_selected_sj = ['weekofyear',
                      'reanalysis_dew_point_temp_k',
                      'reanalysis_relative_humidity_percent',
                      'reanalysis_precip_amt_kg_per_m2',
                      'ndvi_se',
                      'reanalysis_specific_humidity_g_per_kg']

features_selected_iq = ['weekofyear',
                      'reanalysis_min_air_temp_k',
                      'reanalysis_specific_humidity_g_per_kg',
                      'reanalysis_precip_amt_kg_per_m2']

In [8]:
### 4. Back-Testing

# Split the data into a train subset and a test subset
bt_train_samples_sj = 800
bt_train_samples_iq = 400

n_elm_sj = data.loc[(data['city'] == 'sj')].shape[0]
n_elm_iq = data.loc[(data['city'] == 'iq')].shape[0]

bt_train_subset_sj = data.loc[(data['city'] == 'sj')].head(bt_train_samples_sj)
bt_train_subset_iq = data.loc[(data['city'] == 'iq')].head(bt_train_samples_iq)
bt_test_subset_sj = data.loc[(data['city'] == 'sj')].tail(n_elm_sj - bt_train_samples_sj)
bt_test_subset_iq = data.loc[(data['city'] == 'iq')].tail(n_elm_iq - bt_train_samples_iq)

bt_train_features_sj = bt_train_subset_sj[features_selected_sj]
bt_train_features_iq = bt_train_subset_iq[features_selected_iq]
bt_train_labels_sj = bt_train_subset_sj['total_cases']
bt_train_labels_iq = bt_train_subset_iq['total_cases']
bt_test_sj = bt_test_subset_sj[features_selected_sj]
bt_test_iq = bt_test_subset_iq[features_selected_iq]
bt_test_labels_sj = bt_test_subset_sj['total_cases']
bt_test_labels_iq = bt_test_subset_iq['total_cases']

In [9]:
## San Juan

# Normalization of the data
max_abs_scaler = preprocessing.MaxAbsScaler()
bt_train_features_sj = max_abs_scaler.fit_transform(bt_train_features_sj)
bt_test_sj = max_abs_scaler.fit_transform(bt_test_sj)

In [12]:
# Random Forest
n_estimators = 50
max_depth = None
max_features = len(features_selected_sj)

regressor_sj = RandomForestRegressor(n_estimators= n_estimators, max_depth = max_depth, max_features=max_features, criterion='mae', random_state=0)
regressor_sj.fit(bt_train_features_sj, bt_train_labels_sj)

print('Feature Relevancies San Juan')
list1 = zip(features_selected_sj, regressor_sj.feature_importances_)
print(tabulate(list1, headers=['Feature', 'Relevance']))

/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warn(


Feature Relevancies San Juan
Feature                                  Relevance
-------------------------------------  -----------
weekofyear                                0.233308
reanalysis_dew_point_temp_k               0.127984
reanalysis_relative_humidity_percent      0.164388
reanalysis_precip_amt_kg_per_m2           0.155317
ndvi_se                                   0.187915
reanalysis_specific_humidity_g_per_kg     0.131088


In [19]:
# Knn
n_neighbors = 51
weights = 'uniform'
regressor = neighbors.KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=2)
regressor.fit(bt_train_features_sj, bt_train_labels_sj)

KNeighborsRegressor(n_neighbors=51)

In [20]:
# AdaBoost
regressor = AdaBoostRegressor(n_estimators=50)
regressor.fit(bt_train_features_sj, bt_train_labels_sj)

AdaBoostRegressor()

In [21]:
# Gradient Boost
regressor_sj = GradientBoostingRegressor(loss='huber', learning_rate=0.1, n_estimators=100, max_depth=None, criterion='friedman_mse')
regressor_sj.fit(bt_train_features_sj, bt_train_labels_sj)

GradientBoostingRegressor(loss='huber', max_depth=None)

In [22]:
bt_pred_sj = [int(round(x)) for x in regressor_sj.predict(bt_test_sj)]

In [23]:
## Iquitos

# Normalization of the data
max_abs_scaler = preprocessing.MaxAbsScaler()
bt_train_features_iq = max_abs_scaler.fit_transform(bt_train_features_iq)
bt_test_iq = max_abs_scaler.fit_transform(bt_test_iq)

In [24]:
# Knn
n_neighbors = 18
weights = 'distance'

regressor_iq = neighbors.KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=2)
regressor_iq.fit(bt_train_features_iq, bt_train_labels_iq)

KNeighborsRegressor(n_neighbors=18, weights='distance')

In [25]:
# AdaBoost
regressor = AdaBoostRegressor(n_estimators=50)
regressor.fit(bt_train_features_iq, bt_train_labels_iq)

AdaBoostRegressor()

In [26]:
# Random Forest
n_estimators = 50
max_depth = None
max_features = len(features_selected_iq)

In [27]:
regressor_iq = RandomForestRegressor(n_estimators= n_estimators, max_depth = max_depth, max_features=max_features, criterion='mae', random_state=0)
regressor_iq.fit(bt_train_features_iq, bt_train_labels_iq)
print('Feature Relevancies Iquitos')
list2 = zip(features_selected_iq, regressor_iq.feature_importances_)
print(tabulate(list2, headers=['Feature', 'Relevance']))

/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warn(


Feature Relevancies Iquitos
Feature                                  Relevance
-------------------------------------  -----------
weekofyear                                0.227383
reanalysis_min_air_temp_k                 0.187579
reanalysis_specific_humidity_g_per_kg     0.326125
reanalysis_precip_amt_kg_per_m2           0.258914


In [28]:
bt_pred_iq = [int(round(x)) for x in regressor_iq.predict(bt_test_iq)]

In [29]:
## MAE (Mean absolute error)
print("\nBack-Testing - MAE (Mean absolute error):")

mae = mean_absolute_error(bt_test_labels_sj.values.tolist(), bt_pred_sj)
print("\t- San Juan - MAE: %.4f" % mae)

mae = mean_absolute_error(bt_test_labels_iq.values.tolist(), bt_pred_iq)
print("\t- Iquitos  - MAE: %.4f" % mae)

bt_true_labels = bt_test_labels_sj.values.tolist() + bt_test_labels_iq.values.tolist()
bt_pred_labels = bt_pred_sj + bt_pred_iq
mae = mean_absolute_error(bt_true_labels, bt_pred_labels)
print("\tTotal MAE: %.4f" % mae)


Back-Testing - MAE (Mean absolute error):
	- San Juan - MAE: 27.7692
	- Iquitos  - MAE: 7.1638
	Total MAE: 18.0528
